<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Traitement_Donnees/Interrogation_Services_Web.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

# Interrogation d'un service Web (IPA à base de ressources)

## Les marées des sept prochains jours à Tadoussac 

<p>Vous planifiez une sortie en kayak de mer à Tadoussac qui est située sur le fleuve Saint-Laurent à l'embouchure du Saguenay. Vous allez moissonner les informations concernant les marées pour les sept prochains jours à compter du 21 septembre 2022. Pour cela, vous utiliserez un nouveau <a href="https://api-iwls.dfo-mpo.gc.ca/swagger-ui/index.html?configUrl=/v3/api-docs/swagger-config/" target='_blank'>service web</a> avec une interface de programmation à base de ressources (en anglais, REST API) disponible depuis peu sur le site de Pêches et Océans Canada.</p> 

<p>Pour les requêtes HTTP avec le site web, vous utiliserez la bibliothèque python <a href="https://docs.python.org/3/library/urllib.request.html" target='_blank'>urllib.request</a>. L'extraction des sections pertinentes de la page web des résultats sera faite avec l'outil python <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target='_blank'>Beautiful Soup</a>. Enfin, des <a href="https://docs.python.org/fr/3/howto/regex.html" target='_blank'>expressions régulières</a><sup>2</sup> seront utilisées pour extraire les informations détaillées.</p>
<hr/>
<span style="font-size:80%"><sup>1</sup><b>Note - documentation: </b>La documentation sur le <a href="https://api-iwls.dfo-mpo.gc.ca/swagger-ui/index.html?configUrl=/v3/api-docs/swagger-config/" target='_blank'>service web</a> avec une interface de programmation à base de ressources (en anglais, REST API).</span>
<br/>
<span style="font-size:80%"><sup>2</sup><b>Note - pratique: </b><a href="https://regex101.com/" target='_blank'>regex101.com</a> est un excellent site pour pratiquer et mettre au point des expressions régulières.</span>

## Importation des bibliothèques Python
<ul>
<ul>
    <li><b><i>urllib</i></b> (interface client de haut niveau pour HTTP) et <b><i>re</i></b> (expressions régulières), <b><i>json</i></b> (traitement du format de données json: JavaScript Object Notation) et et <b><i>datetime</i></b> (traitement des dates et de l'heure) sont des bibliothèques Python standard, vous n'avez pas besoin de les installer, il suffit de les importer.</li>
  <li>Installation de <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target='_blank'>Beautiful Soup</a></li>
  <ul>
    <li>Par exemple, sudo pip3 install bs4</li>
  </ul>
</ul>
</ul>

In [1]:
# -*- coding: utf-8 -*-
import urllib.request
# sudo pip3 install bs4
from bs4 import BeautifulSoup
import json
import re
import datetime

print("Bibliothèques Python importées")

Bibliothèques Python importées


## Consultation de la documentation l'IPA à base de ressources<sup>1</sup>
La <a href="https://tides.gc.ca/fra/info/WebServicesWLD" target="_blank">documentation sur l'IPA</a> donne un exemple de requête qui vous servira d'inspiration. 

Exemple :

<ul>
    <ol>
        <li>Pour trouver les identifiants (ID) de vos stations d’intérêts : https://api-iwls.dfo-mpo.gc.ca/api/v1/stations.</li>
        <li>Utiliser la requête suivante pour obtenir les niveaux d’eau officiels (wlo) pour la station Vieux-Québec (ID code : 5cebf1e23d0f4a073c4bc0f6) du 1er au 2 octobre 2020 : <span style="background-color:#EFF0F1;">https://api-iwls.dfo-mpo.gc.ca/api/v1/stations/5cebf1e23d0f4a073c4bc0f6/data?time-series-code=wlo&from=2020-10-01T00:00:00Z&to=2020-10-01T00:30:00Z</span></li>
        <li style:font-size:small;>[{"eventDate":"2020-10-01T00:00:00Z","qcFlagCode":"1","value":3.903,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:03:00Z","qcFlagCode":"1","value":3.865,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:06:00Z","qcFlagCode":"1","value":3.825,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:09:00Z","qcFlagCode":"1","value":3.784,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:12:00Z","qcFlagCode":"1","value":3.743,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:15:00Z","qcFlagCode":"1","value":3.7,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:18:00Z","qcFlagCode":"1","value":3.661,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:21:00Z","qcFlagCode":"1","value":3.62,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:24:00Z","qcFlagCode":"1","value":3.578,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:27:00Z","qcFlagCode":"1","value":3.538,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true},<br/>{"eventDate":"2020-10-01T00:30:00Z","qcFlagCode":"1","value":3.495,"timeSeriesId":"5cebf1e23d0f4a073c4bc0e1","reviewed":true}]</li><li>Remplacer l’ID de la station “5cebf1e23d0f4a073c4bc0f6” par celui de votre station d’intérêt;</li>
        <li>Changer la valeur du paramètre “Time-series-code” pour utiliser soit :</li>
        <ul>
            <ul>
                <li>wlo – Niveau d’eau observé</li>
                <li>wlf ou wlf-spine – Prévisions des niveaux d’eau (aux stations opérationnelles seulement)</li>
                <li>wlp – Niveaux d’eau prédits aux 15 minutes</li>
                <li><b>wlp-hilo</b> – Prédictions des hautes et basses mers (<b>Tables de marées</b>)</li>
                <li>Ajuster la période de temps “From” et “To” en utilisant le format ISO 8061 UTC.</li>
            </ul>
        </ul>
    </ol>
</ul>
    
<p>Il suffit donc de remplacer l'id_station par celui de Tadoussac et le code de la série chonologique 'wlo' (water level official)  par celui des marées "wlp-hilo".</p>

<p>Les dates iront du 21 septembre 2023 00:00:00 (soit 0 heure, 0 min, 0 sec) au matin du 28 septembre 2023 à 00:00:00 mais en heure avancée de l'Est (HAE) utilisée au Québec en septembre. Par contre, le temps du service des marées est donné en temps universel (TU). Il faut donc convertir l'heure avancée de l'Est (HAE) utilisée au Québec en temps universel (TU) en ajoutant 4 heures. Donc 00:00:00 HAE à Tadoussac fera 04:00:00 en temps universel (TU). Inversement pour convertir le temps universel (TU) des résultats retournées en heure avancée de l'Est (HAE), il faudra soustraire 4 heures.</p> 
<hr/>
<span style="font-size:80%"><sup>1</sup><b>Note - documentation: </b>La documentation sur le <a href="https://api-iwls.dfo-mpo.gc.ca/swagger-ui/index.html?configUrl=/v3/api-docs/swagger-config/" target='_blank'>service web</a> avec une interface de programmation à base de ressources (en anglais, REST API).</span>


## Programmation d'une requête pour obtenir l'horaire des marées du service Web


### Trouver l'identifiant de la station pour la municipalité de "Tadoussac"

Appeler le service pour obtenir les stations et examiner de la page retournée"

In [2]:
# Obtenir l'IPA à base de ressources de Pêches et Océans
service_web = "https://api-iwls.dfo-mpo.gc.ca/api/v1/"

# Requête pour obtenir la liste des stations
requete = "stations"

with urllib.request.urlopen(service_web+requete) as response:
    page_reponse = response.read()

# Analyser la page de réponse retournée avec BeautifulSoup
dom_page_resultats = BeautifulSoup(page_reponse,"html.parser") 
# Afficher la page web retournée
print(dom_page_resultats.prettify())


[{"id":"5cebf1e13d0f4a073c4bbed3","code":"01025","officialName":"Dover - Wellington","operating":false,"latitude":48.866667,"longitude":-53.966667,"type":"DISCONTINUED","timeSeries":[{"id":"5cebf1e13d0f4a073c4bbed2","code":"wlp","nameEn":"Water level predictions","nameFr":"Prédictions de niveaux d'eau","phenomenonId":"5ce598df487b84486892821c"},{"id":"5d9dd7cc33a9f593161c4009","code":"wlp-hilo","nameEn":"High and Low Tide Predictions","nameFr":"Prédictions de pleines et basses mers","phenomenonId":"5ce598df487b84486892821c"},{"id":"5dd306f0e0fdc4b9b4bfa377","code":"wlo","nameEn":"Water level official value","nameFr":"Niveau d'eau, valeur officielle","phenomenonId":"5ce598df487b84486892821c"}]},{"id":"5cebf1df3d0f4a073c4bbcc9","code":"00514","officialName":"Ecum Secum","operating":false,"latitude":44.966667,"longitude":-62.133333,"type":"DISCONTINUED","timeSeries":[{"id":"5cebf1df3d0f4a073c4bbcc8","code":"wlp","nameEn":"Water level predictions","nameFr":"Prédictions de niveaux d'eau","p

### Extraction de l'identifiant et du code de la station

Dans le code HTML de la page, on retrouve la section contenant le mot «Tadoussac». On en examine un bout pour en extraire le contenu en utilisant une expression régulière. 

`{"id":"5cebf1df3d0f4a073c4bbbaf","code":"03425","officialName":"Tadoussac"`

Évidemment pour faire un simple test, on pourrait se contenter d'extraire l'identifiant (5cebf1df3d0f4a073c4bbbaf) manuellement. L'extraction par programmation au moyen d'une expression régulière permettra d'automatiser la tâche pour une éventuelle réutilisation. Voir la <a href="https://docs.python.org/fr/3/howto/regex.html" target='_blank'>documentation française sur les expressions régulières</a> en Python. Aidez-vous du site <a href="https://regex101.com/" target='_blank'>regex101.com</a> pour mettre au point votre expression régulière. 

In [3]:
municipalite_requise = "Tadoussac"

FORME = re.compile(r'"id":"([\w|\d]*)","code":"(\d*)","officialName":"'+municipalite_requise)
formes_reconnues = re.search(FORME,dom_page_resultats.get_text())
if formes_reconnues:
    id_station = formes_reconnues.group(1)
    code_station = formes_reconnues.group(2)
print("id_station:",id_station)
print("code_station:",code_station)

id_station: 5cebf1df3d0f4a073c4bbbaf
code_station: 03425


### Programmation de la requête pour obtenir l'horaire des marées

Vous construisez la requête en modifiant l'exemple `https://api-iwls.dfo-mpo.gc.ca/api/v1/stations/5cebf1e23d0f4a073c4bc0f6/data?time-series-code=wlo&from=2020-10-01T00:00:00Z&to=2020-10-01T00:30:00Z`

In [ ]:
service_web = "https://api-iwls.dfo-mpo.gc.ca/api/v1/"

id_station = "5cebf1df3d0f4a073c4bbbaf"
date_requise_debut = "2022-09-21"
heure_requise_debut = "04:00:00"
date_requise_fin = "2022-09-28"
heure_requise_fin = "04:00:00"

requete = "stations/" + id_station + "/data?time-series-code=wlp-hilo&from=" + \
          date_requise_debut + "T" + heure_requise_debut + "Z&to=" + \
          date_requise_fin + "T" + heure_requise_fin + "Z"

with urllib.request.urlopen(service_web+requete) as response:
    page_reponse = response.read()

# Récupération de la page de résultats au moyen de BeautifulSoup
dom_page_resultats = BeautifulSoup(page_reponse,"html.parser") 

# Conversion et affichage en format JSON
# essentiellement pour avoir un affichage pratique
json_page_resultats = json.loads(dom_page_resultats.get_text())
for element in json_page_resultats:
    print(element)


### Extraction des informations sur les marées

L'idée est de produire exactement le même fichier de données en .csv que celui obtenu par moissonnage du formulaire web. Voir le laboratoire précédent. Pour cela, il faut convertir l'heure données en temps universel (TU) en heure avancée de l'st (HAE). 

In [8]:
import datetime

def convertir_TU_HAE(entree_TU):
    # Extraction des paramètres pour la date et l'heure 
    # d'une chaîne de caractères du genre '2022-09-21T07:37:00Z'
    FORME2 = re.compile('(\d*)-(\d*)-(\d*)T(\d*):(\d*):(\d*)Z')
    formes_reconnues = re.search(FORME2,entree_TU)
    if formes_reconnues:
        annee = int(formes_reconnues.group(1))
        mois = int(formes_reconnues.group(2))
        jour = int(formes_reconnues.group(3))
        heure = int(formes_reconnues.group(4))
        minute = int(formes_reconnues.group(5))
        seconde = int(formes_reconnues.group(6))        
        date_et_heure = datetime.datetime(annee, mois, jour, heure, minute, seconde)
        # Conversion TU vers HAE par la soustraction de 4 heures
        temps_ajoute = datetime.timedelta(hours=4)
        nouveau_date_et_heure = date_et_heure - temps_ajoute
    # Retour d'une chaine de 
    return str(nouveau_date_et_heure) 

### Extraction des informations sur les marées

On extrait l'information utile sur les marées comme la date, l'heure convertie en HAE et la hauteur de la marée. Les autres informations comme `qcFlagCode` et `timeSeriesId` sont mises de côté. 

In [9]:
donnees_marees_liste = []

for evenement_maree in json_page_resultats:
    temps_HAE = convertir_TU_HAE(evenement_maree['eventDate'])
    hauteur_m = evenement_maree['value'] 
    donnees_marees_liste.append(temps_HAE.replace(' ','\t')+'\t'+str(hauteur_m))
print(donnees_marees_liste)

['2022-09-21\t00:23:00\t3.582', '2022-09-21\t07:10:00\t1.358', '2022-09-21\t13:11:00\t3.035', '2022-09-21\t18:55:00\t1.53', '2022-09-22\t01:20:00\t3.791', '2022-09-22\t07:51:00\t1.14', '2022-09-22\t13:54:00\t3.313', '2022-09-22\t19:42:00\t1.246', '2022-09-23\t02:02:00\t4.014', '2022-09-23\t08:24:00\t0.916', '2022-09-23\t14:28:00\t3.601', '2022-09-23\t20:21:00\t0.953', '2022-09-24\t02:38:00\t4.216', '2022-09-24\t08:54:00\t0.705', '2022-09-24\t14:59:00\t3.886', '2022-09-24\t20:57:00\t0.676', '2022-09-25\t03:10:00\t4.377', '2022-09-25\t09:23:00\t0.52', '2022-09-25\t15:29:00\t4.157', '2022-09-25\t21:32:00\t0.439', '2022-09-26\t03:43:00\t4.479', '2022-09-26\t09:52:00\t0.381', '2022-09-26\t15:59:00\t4.395', '2022-09-26\t22:08:00\t0.266', '2022-09-27\t04:16:00\t4.501', '2022-09-27\t10:22:00\t0.308', '2022-09-27\t16:31:00\t4.579', '2022-09-27\t22:44:00\t0.18']


## Sauvegarde des données dans le fichier `donnees_marees_service_web.csv`

Le fichier de données en format .csv est reconstitué à l'identique avec le fichier généré dans le laboratoire précédent.

In [10]:
# Sauvegarder les données dans un fichier .csv
chemin_fichier_sortie = './'
nom_fichier_sortie = "donnees_marees_service_web.csv"
municipalite_requise = 'Tadoussac'
fuseau_horaire_requis = 'HAE'

with open(chemin_fichier_sortie+nom_fichier_sortie,'w') as fichier_sortie:
    # Écriture de l'entête du fichier listant les différents attributs
    fichier_sortie.write('localite\tstation\tfuseau_horaire\tdate\theure\thauteur_m\n')
    for donnees_marees in donnees_marees_liste:
        # Écriture des données sur la marée dans le fichier
        fichier_sortie.write(municipalite_requise+'\t'+code_station+'\t'+fuseau_horaire_requis+'\t'+donnees_marees+'\n')
        
print("Données de marées sauvegardées dans le fichier "+ nom_fichier_sortie)


Données de marées sauvegardées dans le fichier donnees_marees_service_web.csv


## Test de lecture du fichier de données

Vérification que le fichier .csv est correct.

In [11]:
import pandas as pd

donnees_marees_df = pd.read_csv(chemin_fichier_sortie+nom_fichier_sortie,delimiter='\t')
donnees_marees_df

,localite,station,fuseau_horaire,date,heure,hauteur_m
0,Tadoussac,3425,HAE,2022-09-21,00:23:00,3.582
1,Tadoussac,3425,HAE,2022-09-21,07:10:00,1.358
2,Tadoussac,3425,HAE,2022-09-21,13:11:00,3.035
3,Tadoussac,3425,HAE,2022-09-21,18:55:00,1.530
4,Tadoussac,3425,HAE,2022-09-22,01:20:00,3.791
5,Tadoussac,3425,HAE,2022-09-22,07:51:00,1.140
6,Tadoussac,3425,HAE,2022-09-22,13:54:00,3.313
7,Tadoussac,3425,HAE,2022-09-22,19:42:00,1.246
8,Tadoussac,3425,HAE,2022-09-23,02:02:00,4.014
9,Tadoussac,3425,HAE,2022-09-23,08:24:00,0.916
